In [1]:
#Hello
import pandas as pd
import string
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn_pandas import DataFrameMapper
from xgboost import XGBClassifier
from sklearn import preprocessing

In [2]:
df = pd.read_csv('train.csv', index_col="Loan_ID")

In [3]:
df.head()

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender,Interest_Rate
Loan_ID,,,,,,,,,,,,,
10000001,"7,000",< 1 year,Rent,68000.0,not verified,car,18.37,0,NaN,9,14,Female,1
10000002,"30,000",4 years,Mortgage,NaN,VERIFIED - income,debt_consolidation,14.93,0,17.0,12,24,Female,3
10000003,"24,725",7 years,Mortgage,75566.4,VERIFIED - income source,debt_consolidation,15.88,0,NaN,12,16,Male,3
10000004,"16,000",< 1 year,NaN,56160.0,VERIFIED - income source,debt_consolidation,14.34,3,NaN,16,22,Male,3
10000005,"17,000",8 years,Own,96000.0,VERIFIED - income source,debt_consolidation,22.17,1,NaN,19,30,Female,1


In [4]:
df.isna().sum()

Loan_Amount_Requested          0
Length_Employed             7371
Home_Owner                 25349
Annual_Income              25102
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    88379
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
Interest_Rate                  0
dtype: int64

#### Handling missing data

In [5]:
def cleaning_data(df):
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].str.replace(",","")
    df["Loan_Amount_Requested"] = df["Loan_Amount_Requested"].astype(float)
    df["Length_Employed"] = df["Length_Employed"].str.replace(r'[><+]+',"")
    df["Length_Employed"] = df["Length_Employed"].str.extract('(\d+)').astype(float)
    df["Length_Employed"] = df["Length_Employed"].transform(lambda x: x.fillna(-1))
    df["Annual_Income"] = df.groupby(["Length_Employed"])["Annual_Income"].transform(lambda x: x.fillna(x.median()))
    df["Home_Owner"] = df["Home_Owner"].transform(lambda x: x.fillna("-1"))
    df["Months_Since_Deliquency"] = df["Months_Since_Deliquency"].transform(lambda x: x.fillna(-1))
    return df

In [6]:
df = cleaning_data(df)

In [7]:
from scipy import stats
def normalize_data(df):
    df["Loan_Amount_Requested"] = stats.zscore(df["Loan_Amount_Requested"], nan_policy='omit')
    df["Length_Employed"] = stats.zscore(df["Length_Employed"], nan_policy='omit')
    df["Annual_Income"] = stats.zscore(df["Annual_Income"], nan_policy='omit')
    df["Months_Since_Deliquency"] = stats.zscore(df["Months_Since_Deliquency"], nan_policy='omit')
    df["Debt_To_Income"] = stats.zscore(df["Debt_To_Income"], nan_policy='omit')
    df["Number_Open_Accounts"] = stats.zscore(df["Number_Open_Accounts"], nan_policy='omit')
    df["Total_Accounts"] = stats.zscore(df["Number_Open_Accounts"], nan_policy='omit')
    df["Inquiries_Last_6Mo"] = stats.zscore(df["Inquiries_Last_6Mo"], nan_policy='omit')
    return df

In [8]:
original_df = df.copy()
df = df.drop(["Interest_Rate"], axis=1)

In [9]:
scaled_features_df = normalize_data(df)

In [10]:
scaled_features_df

,Loan_Amount_Requested,Length_Employed,Home_Owner,Annual_Income,Income_Verified,Purpose_Of_Loan,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Gender
Loan_ID,,,,,,,,,,,,
10000001,-0.887404,-1.277968,Rent,-0.066557,not verified,car,0.148222,-0.755451,-0.708951,-0.439484,-0.439484,Female
10000002,1.889756,-0.472088,Mortgage,-0.210036,VERIFIED - income,debt_consolidation,-0.290270,-0.755451,0.074897,0.161501,0.161501,Female
10000003,1.252820,0.333792,Mortgage,0.069146,VERIFIED - income source,debt_consolidation,-0.169175,-0.755451,-0.708951,0.161501,0.161501,Male
10000004,0.199311,-1.277968,-1,-0.278907,VERIFIED - income source,debt_consolidation,-0.365477,2.143818,-0.708951,0.962816,0.962816,Male
10000005,0.320057,0.602418,Own,0.435622,VERIFIED - income source,debt_consolidation,0.632603,0.210972,-0.708951,1.563802,1.563802,Female
...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.769836,1.139671,Rent,-0.030687,VERIFIED - income source,credit_card,-0.464903,-0.755451,-0.708951,0.361830,0.361830,Male
10164306,-1.008150,0.602418,Own,-0.156232,VERIFIED - income source,small_business,-1.030864,-0.755451,0.902293,0.361830,0.361830,Male
10164307,0.712482,1.139671,Mortgage,-0.030687,VERIFIED - income,credit_card,0.122728,0.210972,-0.708951,1.163144,1.163144,Male


In [11]:
def label_encode(df):
    # le = preprocessing.LabelEncoder()
    # df["Income_Verified"] = le.fit_transform(df["Income_Verified"])
    # df["Home_Owner"] = le.fit_transform(df["Home_Owner"])
    # df["Purpose_Of_Loan"] = le.fit_transform(df["Purpose_Of_Loan"])
    # df["Gender"] = le.fit_transform(df["Gender"])
    #onehotlabels = pd.get_dummies(df)
    onehotlabels = pd.get_dummies(df, prefix=None, prefix_sep='_', 
                       dummy_na=True, columns=["Income_Verified","Home_Owner","Purpose_Of_Loan","Gender"], sparse=False, 
                       drop_first=False, dtype=None)
    #onehotlabels = np.array(onehotlabels.values)
    return onehotlabels

In [12]:
scaled_features_df = label_encode(scaled_features_df)

In [13]:
# encoded = pd.DataFrame(encoded)

In [14]:
# scaled_features_df = scaled_features_df.append(encoded)

In [15]:
scaled_features_df

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,...,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Purpose_Of_Loan_nan,Gender_Female,Gender_Male,Gender_nan
Loan_ID,,,,,,,,,,,,,,,,,,,,,
10000001,-0.887404,-1.277968,-0.066557,0.148222,-0.755451,-0.708951,-0.439484,-0.439484,0,0,...,0,0,0,0,0,0,0,1,0,0
10000002,1.889756,-0.472088,-0.210036,-0.290270,-0.755451,0.074897,0.161501,0.161501,1,0,...,0,0,0,0,0,0,0,1,0,0
10000003,1.252820,0.333792,0.069146,-0.169175,-0.755451,-0.708951,0.161501,0.161501,0,1,...,0,0,0,0,0,0,0,0,1,0
10000004,0.199311,-1.277968,-0.278907,-0.365477,2.143818,-0.708951,0.962816,0.962816,0,1,...,0,0,0,0,0,0,0,0,1,0
10000005,0.320057,0.602418,0.435622,0.632603,0.210972,-0.708951,1.563802,1.563802,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.769836,1.139671,-0.030687,-0.464903,-0.755451,-0.708951,0.361830,0.361830,0,1,...,0,0,0,0,0,0,0,0,1,0
10164306,-1.008150,0.602418,-0.156232,-1.030864,-0.755451,0.902293,0.361830,0.361830,0,1,...,0,0,0,1,0,0,0,0,1,0
10164307,0.712482,1.139671,-0.030687,0.122728,0.210972,-0.708951,1.163144,1.163144,1,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
X_features = scaled_features_df

In [17]:
X_features

,Loan_Amount_Requested,Length_Employed,Annual_Income,Debt_To_Income,Inquiries_Last_6Mo,Months_Since_Deliquency,Number_Open_Accounts,Total_Accounts,Income_Verified_VERIFIED - income,Income_Verified_VERIFIED - income source,...,Purpose_Of_Loan_moving,Purpose_Of_Loan_other,Purpose_Of_Loan_renewable_energy,Purpose_Of_Loan_small_business,Purpose_Of_Loan_vacation,Purpose_Of_Loan_wedding,Purpose_Of_Loan_nan,Gender_Female,Gender_Male,Gender_nan
Loan_ID,,,,,,,,,,,,,,,,,,,,,
10000001,-0.887404,-1.277968,-0.066557,0.148222,-0.755451,-0.708951,-0.439484,-0.439484,0,0,...,0,0,0,0,0,0,0,1,0,0
10000002,1.889756,-0.472088,-0.210036,-0.290270,-0.755451,0.074897,0.161501,0.161501,1,0,...,0,0,0,0,0,0,0,1,0,0
10000003,1.252820,0.333792,0.069146,-0.169175,-0.755451,-0.708951,0.161501,0.161501,0,1,...,0,0,0,0,0,0,0,0,1,0
10000004,0.199311,-1.277968,-0.278907,-0.365477,2.143818,-0.708951,0.962816,0.962816,0,1,...,0,0,0,0,0,0,0,0,1,0
10000005,0.320057,0.602418,0.435622,0.632603,0.210972,-0.708951,1.563802,1.563802,0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10164305,0.769836,1.139671,-0.030687,-0.464903,-0.755451,-0.708951,0.361830,0.361830,0,1,...,0,0,0,0,0,0,0,0,1,0
10164306,-1.008150,0.602418,-0.156232,-1.030864,-0.755451,0.902293,0.361830,0.361830,0,1,...,0,0,0,1,0,0,0,0,1,0
10164307,0.712482,1.139671,-0.030687,0.122728,0.210972,-0.708951,1.163144,1.163144,1,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
y = original_df["Interest_Rate"]

In [19]:
y

Loan_ID
10000001    1
10000002    3
10000003    3
10000004    3
10000005    1
           ..
10164305    2
10164306    2
10164307    1
10164308    3
10164309    2
Name: Interest_Rate, Length: 164309, dtype: int64

In [20]:
#import XGBoost classifier and accuracy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#instantiate model and train
#learning_rate = 0.09, n_estimators=500, max_depth=5
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              learning_rate=0.04, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=-1,
              nthread=None, objective='multi:softmax', random_state=0)
model.fit(X_features, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.04, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=1000, n_jobs=-1, nthread=-1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [21]:
test_df = pd.read_csv("test.csv", index_col = "Loan_ID")

In [22]:
test_df.isna().sum()

Loan_Amount_Requested          0
Length_Employed             4936
Home_Owner                 16711
Annual_Income              16898
Income_Verified                0
Purpose_Of_Loan                0
Debt_To_Income                 0
Inquiries_Last_6Mo             0
Months_Since_Deliquency    58859
Number_Open_Accounts           0
Total_Accounts                 0
Gender                         0
dtype: int64

In [23]:
test_df = cleaning_data(test_df)

In [24]:
test_df = label_encode(test_df)

In [25]:
scaled_features_test_df = normalize_data(test_df)

In [26]:
predictions = model.predict(scaled_features_test_df)

In [27]:
output_df = pd.DataFrame({'Loan_ID': scaled_features_test_df.index, 'Interest_Rate': predictions})

In [28]:
output_df.to_csv("solution_final.csv", index=False)